In [62]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
stocks=['MSFT','AAPL','NVDA','GOOGL','AMZN','META','BRK-B','LLY','AVGO','V','JPM','TSLA','WMT','XOM','UNH','MA','PG','JNJ','HD','ORCL','MRK','COST','ABBV','CVX','BAC','CRM','NFLX','KO','AMD','PEP','ADBE','TMO','DIS','WFC','MCD','CSCO','TMUS','ABT','QCOM','CAT','DHR','INTU','GE','IBM','VZ','AMAT','AXP','CMCSA','NOW','COP','INTC','TXN','UBER','BX','MS','PFE','NKE','AMGN','PM','UNP','RTX','ISRG','SPGI','GS','LOW','NEE','MU','SCHW','SYK','HON','PGR','UPS','LRCX','ELV','BKNG','T','BLK','C','DE','LMT','TJX','BA','ABNB','VRTX','BSX','ADP','PLD','CI','SBUX','REGN','MMC','BMY','ADI','PANW','MDLZ','KLAC','SCCO','FI','CVS','DELL','KKR','GILD','WM','HCA','ANET','SNPS','AMT','CMG','CDNS','SHW','GD','EOG','SO','TGT','CME','ITW','ICE','MPC','DUK','MO','SLB','FCX','CL','CRWD','ZTS','EQIX','PH','MCK','MAR','MCO','TDG','CTAS','WDAY','PSX','BDX','APH','NOC','CSX','PYPL','FDX','ORLY','EMR','ECL','PXD','USB','EPD','APO','PCAR','RSG','PNC','OXY','CEG','MRVL','MSI','MNST','ROP','SMCI','VLO','NSC','DASH','EW','COF','CPRT','COIN','DXCM','ET','WELL','APD','AZO','HLT','MMM','AJG','MET','SNOW','EL','AIG','FTNT','GM','CARR','DHI','COR','TFC','TRV','STZ','F','GWW','NUE','HES','AFL','PSA','IBKR','ADSK','MCHP','SPG','WMB','ODFL','OKE','SQ','PLTR']
stocks=[stock.replace('-','.') for stock in stocks][:100]

In [64]:
from order_routerv2 import *
import tensorflow as tf
from keras.models import load_model
from utils import*
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor
from datetime import*
import pickle as pkl


from alpaca.data.historical import *
from alpaca.data.requests import *
from alpaca.data.timeframe import *
from alpaca.data.enums import *
from alpaca.trading.client import *
from alpaca.trading.requests import * 
from alpaca.trading.enums import *

with open('multi_scaler3.pkl','rb') as f:
    multi_scaler=pkl.load(f)
with open('encoder.pkl','rb') as f:
    encoder=pkl.load(f)
    


In [59]:
trader=Trader(model=load_model('model3.keras'),multi_scaler=multi_scaler,encoder=encoder)


In [119]:
bars.groupby('symbol').last()['close']

symbol
AAPL    169.60
ABBV    159.62
ABNB    164.01
ABT     107.50
ADBE    477.64
         ...  
VRTX    397.48
VZ       39.67
WFC      59.96
WMT      60.16
XOM     118.04
Name: close, Length: 100, dtype: float64

In [121]:
bars=trader.get_bars(stocks,datetime.now())
arrs=trader.preprocess_bars_v2(bars,stocks)
predictions=trader.predict(arrs)
trades=dict(zip(stocks,predictions))


1/1 [==============================] - 0s 404ms/step
[1 1 0 1 0 0 1 1 0 0 1 0 1 1 1 0 1 1 0 1 1 0 0 1 1 0 0 1 0 1 1 0 0 1 1 1 0
 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1]
0.76% long


TypeError: 'type' object is not iterable

In [103]:
predictions

array([1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1])

In [81]:
import multiprocessing
    with multiprocessing.Pool(int(multiprocessing.cpu_count())) as p:
    results = [p.apply_async(preprocess,args=[]) for x in ]
    
    for result in results:
        result.get()
        




IndentationError: unexpected indent (4101356607.py, line 2)

In [92]:
res = supabase.table('DataStream') \
    .select('S','o','h','l','c','v','t') \
    .in_('S',stocks) \
    .gte('t',datetime.now()-timedelta(days=37)) \
    .lte('t',datetime.now()) \
    .order('t',desc=False) \
    .execute()
bars=pd.DataFrame(res.model_dump()['data']).rename(columns={'S':'symbol','o':'open','h':'high','l':'low','c':'close','v':'volume','t':'timestamp'}).assign(timestamp=lambda x:pd.to_datetime(x.timestamp)).set_index(['symbol','timestamp']).sort_index()
bars

open    high     low    close    volume
symbol timestamp                                                            
AAPL   2024-04-12 20:00:00+00:00  176.570  176.74  176.54  176.700  11196573
       2024-04-12 20:15:00+00:00  176.740  176.92  176.55  176.900     67294
       2024-04-12 20:30:00+00:00  176.930  176.94  176.55  176.910    102702
       2024-04-12 20:45:00+00:00  176.920  176.92  176.85  176.910    104037
       2024-04-12 21:00:00+00:00  176.910  176.99  176.88  176.910     22518
...                                   ...     ...     ...      ...       ...
TSLA   2024-04-17 19:15:00+00:00  156.690  157.15  156.21  156.250   2126073
       2024-04-17 19:30:00+00:00  156.260  156.58  155.83  156.320   2058991
       2024-04-17 19:45:00+00:00  156.310  156.46  155.30  155.425   3558174
       2024-04-17 20:00:00+00:00  155.450  155.56  154.60  154.748   2934872
       2024-04-17 20:15:00+00:00  154.747  155.45  154.69  154.700     47794

[907 rows x 5 columns]

In [73]:
trader.client.get_stock_latest_quote(StockLatestQuoteRequest(symbol_or_symbols='AAPL',feed='sip'))


{'AAPL': {   'ask_exchange': 'P',
     'ask_price': 169.7,
     'ask_size': 11.0,
     'bid_exchange': 'P',
     'bid_price': 169.6,
     'bid_size': 17.0,
     'conditions': ['R'],
     'symbol': 'AAPL',
     'tape': 'C',
     'timestamp': datetime.datetime(2024, 4, 26, 23, 59, 32, 341086, tzinfo=TzInfo(UTC))}}

In [74]:
bars

""
